# Tests4Py Statistical Fault Localization

In this notebook we demonstrate the usage of Tests4Py in the context of statistical fault localization. 
We will use the `middle_2` subject as an example in this tutorial.

## Importing the API

To get access to the API, we need to import the `tests4py` module.

In [1]:
from pathlib import Path

import tests4py.api as t4p
from tests4py import sfl

## Retrieving the Subject

As a next step we can retrieve the subject, such that we have a local copy of it to work with.

In [2]:
report = t4p.checkout(t4p.middle_2)

We will use the `report` to extract the original source since fault localization may need it later on.

In [3]:
src = report.location

## Instrumenting the Subject

Now we can instrument the subject to retrieve the fault localization information. 
This will also install all dependencies and the subject in a virtual environment. 
It may also install the correct Python version, if it is not already installed (Note that this may take a while).

In [4]:
dst = Path("tmp", "sfl")
sfl.sflkit_instrument(dst, events="line")

## Executing the Relevant Tests

Let's execute the relevant test cases to get the fault localization information.

In [6]:
sfl.sflkit_unittest(work_dir=dst)

## Analyzing the Fault Localization Information

Now we can leverage the fault localization information to analyze the subject and find the most likely faulty lines.

In [7]:
report = sfl.sflkit_analyze(work_dir=dst, 
                            src=src,
                            predicates="line")

Now we retrieve the suggested lines based on the Ochiai metric.

In [8]:
from sflkit.analysis.analysis_type import AnalysisType
from sflkit.analysis.spectra import Spectrum

suggestions = report.analyzer.get_sorted_suggestions(src,
            type_=AnalysisType.LINE,
            metric=Spectrum.Ochiai,)
suggestions

## Evaluation

With Tests4Py we can get the faulty lines, i.e. the lines that were changed with the fix.

In [9]:
faulty_lines = t4p.get_faulty_lines(t4p.middle_2)
faulty_lines

As you can see, the top suggested line is indeed the correctly evaluated faulty line.

In [10]:
assert faulty_lines[0] in suggestions[0].lines

## Summary

Tests4Py is with its direct integration of fault localization tools a powerful tool to analyze approaches
in this area.